In [3]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials

import pandas as pd
import psycopg2 as pg
import sqlalchemy as sql
import os
os.environ['SPOTIPY_CLIENT_ID']='3aa61ed19d084b3d996991f3bb3d0b3e'
os.environ['SPOTIPY_CLIENT_SECRET']='7a273c762d324a4698f3364ae07f9da7'

birdy_uri = 'spotify:artist:2WX2uTcsvV5OnS0inACecP'
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

results = spotify.artist_albums(birdy_uri, album_type='album')
albums = results['items']
while results['next']:
    results = spotify.next(results)
    albums.extend(results['items'])

for album in albums:
    print(album['name'])

Beautiful Lies
Beautiful Lies
Beautiful Lies (Deluxe)
Beautiful Lies (Deluxe)
Fire Within
Fire Within
Fire Within (Deluxe)
Fire Within (Deluxe)
Fire Within (Deluxe)
Live in London
Birdy
Birdy
Birdy
Birdy
Birdy (Deluxe Version)


In [5]:
redirect_uri = 'http://localhost:8080'
scope = "user-library-read"

auth = spotipy.oauth2.SpotifyOAuth(
    client_id=os.environ['SPOTIPY_CLIENT_ID'],
    client_secret=os.environ['SPOTIPY_CLIENT_SECRET'],
    redirect_uri=redirect_uri
)
sp = spotipy.Spotify(auth=auth)
results = sp.current_user()

HTTP Error for GET to https://api.spotify.com/v1/me/ returned 400 due to Only valid bearer authentication supported


SpotifyException: http status: 400, code:-1 - https://api.spotify.com/v1/me/:
 Only valid bearer authentication supported, reason: None

In [ ]:
def get_seed_artists(sp):
    playlists = []
    artists = []
    results = sp.current_user_playlists()
    for idx, item in enumerate(results['items']):
        playlists.append(item.get('playlist')
    for play in playlists:
        art_results = sp.playlist(play, fields='artist')
        for art_idx, art_item in enumerate(art_results['items']):
            artists.append(art_item.get('artist')
        artists = list(set(artists))
    return artists

In [ ]:
def get_album_tracks(sp, artists):
    tracks, albums = [], []
    for artist in artists:
        results = sp.artist_albums(artist, country='US', album_type='album', limit=20)
        for idx, item in enumerate(results['items']):
            tracks.append(item.get('track'))
    tracks = list(set(tracks))
    return tracks

In [ ]:
def get_track_info(sp, tracks):
    track_info = []
    num = 0
    while num <= len(tracks):
        num +=100
        track_info.extend(sp.audio_features(tracks[:num]))
    return track_info

In [ ]:
def main(sp):
    artists=get_seed_artists(sp)
    tracks=get_album_tracks(sp, artists)
    track_info=get_track_info(sp, tracks)
    
#convert track_info to df
    push_to_db(df) #if engine and function are already defined

In [ ]:
engine = sql.create_engine('postgresql+psycopg2://<username>:<password>@localhosts:<port>/<db_name>')

In [ ]:
def push_to_db(df):
    df.to_sql('table_name', engine, schema='public', if_exists='append', index=False)
    return